In [1]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoTokenizer, TrainingArguments, Trainer, GenerationConfig
from transformers import BitsAndBytesConfig, HfArgumentParser
from tqdm import tqdm
from trl import SFTTrainer 
import torch
import time 
import os
import pandas as pd 
import numpy as np 
from datasets import Dataset
from huggingface_hub import interpreter_login
from random import randrange
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from sklearn.model_selection import train_test_split

In [2]:
default_path = os.getcwd()
data_path = os.path.join(default_path, '../../data')
model_path = os.path.join(default_path, '../../models')
local_mistral_dir = os.path.join(model_path, "mistral_origin")
config_path = os.path.join(default_path, '../../config')

In [3]:
com_data = load_dataset("MarkrAI/KoCommercial-Dataset", split='train') 

In [4]:
len(com_data)

1444372

In [5]:
com_data

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 1444372
})

In [6]:
qa_idx = [] 
for idx, val in enumerate(com_data['input']):
    # print(val)
    if '답변' in val and '원문' in val:
        qa_idx.append(idx)

In [7]:
gen_idx = [] 
for idx, val in enumerate(com_data['input']):
    # print(val)
    if '문장' in val and '생성' in val:
        gen_idx.append(idx)

In [8]:
set(com_data['input']), len(qa_idx)

({'',
  '당신은 인공지능 비서입니다. 주어진 문장 리스트를 모두 활용하여 가장 정확한 단락을 생성하세요.',
  '당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.',
  '당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.',
  '당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.',
  '솔루션으로 A, B, C 또는 D를 선택합니다.'},
 135495)

In [9]:
com_df = pd.DataFrame(zip(com_data['input'], com_data['instruction'], com_data['output']), columns=['input', 'instruction', 'output'])
com_df

,input,instruction,output
0,,"보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너...","모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려..."
1,,저희 학교 수학 클럽에는 남학생 6명과 여학생 8명이 있습니다. 주 수학 경시대회...,14명 중 6명을 선택해야 하는데 순서는 중요하지 않습니다. 이것은 순열 문제가 아...
2,,"자음이 하나 이상인 4글자 단어는 $A$, $B$, $C$, $D$, $E$로 몇 ...",먼저 단어에 제한을 두지 않고 4글자로 된 모든 단어의 개수를 세어봅니다. 그런 다...
3,,멜린다는 표준 6면 주사위 두 개를 굴려서 굴린 두 개의 숫자로 두 자리 숫자를 만...,주사위 중 하나 이상이 1에 나올 때만 가능합니다. 두 주사위가 모두 1이 아닐 확...
4,,p$를 공정한 동전을 반복적으로 던지는 과정에서 5$의 앞면이 나오기 전에 2$의 ...,문제를 H와 T의 시퀀스라고 생각하세요. 두 개의 T가 연속으로 나타날 수 없으므로...
...,...,...,...
1444367,당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.,"이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들...",이번 추석에도 멀리 해외에 계신 동포 여러분께 안부 인사를 전합니다.
1444368,당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.,"이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들...",옷이 너무 비싸서 친구의 선물로 살지 말지 고민이다.
1444369,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,아무래도
1444370,당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.,"이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들...","검은색, 붉은색, 녹색의 다채로운 쌀과, 찹쌀 등을 생산하고 있다."


In [10]:
qa_df = com_df.iloc[qa_idx]
qa_df.reset_index(inplace=True, drop=True)
print(qa_df['instruction'][1000], end='\n\n')
print(qa_df['output'][1000])

다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞게 완성하세요. 문장은 이전 내용과 논리적으로 연결되어야 합니다.
#텍스트: 왕머루 화형에 따른 암머루×암머루, 암머루×포도의 상호 교잡의 경우, GW-45×GW-202 또는 GW-202×GW-45의 상호역교배 시 자가수분과 같이 착과된 과립이 전혀 없었으며 단위결과성 소립과 형태의 착과도 없었고 화경이 말라서 소멸됨에 따라서 이들 암머루간 상호역 교배 시 착과가 전혀 이루어지지 않았다(Table 4). 또한 ‘Buffalo’와 ‘MBA’에 각각의 GW-202의 화분을 교배한 결과 소립과 형태의 착과율는 8.5%, 5.4%로 나타났으나 이들 소과립 내의 종자가 있는 과립은 전혀 없었던 것으로 나타난 것으로 보아 이들 소과립과는 포도품종의 위단위결과성에 의해 착과된 것으로 생각된다(Table 5). 또한 GW-202×‘Buffalo’, 교잡의 경우, 착과율 23.5%, 1과립당 종자수 2.5개, 정상 종자형성도 97.0%로 암머루와 포도와도 교배 친화성이 있는 것으로 나타났다.

따라서 암머루×암머루, 포도×암머루의 인공수분 결과 정상적인 종자를 전혀 획득할 수 없었던 것은 암머루의 화분은 SEM 관찰 결과인 acolporated 형태로 비정상적 화분에 의한 화분발아가 전혀 이루어지지 않는 것으로 조사되었고, 특히 ‘Buffalo’와 ‘MBA’이 포도 품종은 위단위결과성에 의해 소립과가 착과되었으나(Stout, 1936) 암머루는 자가수분이나 암머루 상호교잡에서 전혀 착과가 없는 것은 위단위결과성이 전혀 없는 종으로 생각된다.


In [11]:
qa_df.head(1)

,input,instruction,output
0,당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변...,"다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞...","그래서 문학은 가옥의 배열이나 마을의 배치를 설명하거나, 귀신을 내쫓고 운명을 예견..."


In [12]:
compute_dtype = getattr(torch, 'float16')
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=compute_dtype, 
    bnb_4bit_use_double_quant=False,
)

In [13]:
model_dir = os.path.join(model_path, "mistral_origin")

In [14]:
tokenizer = AutoTokenizer.from_pretrained(os.path.join(model_dir, 'tokenizer'))
tokenizer.pad_token

'</s>'

In [15]:
tokenizer.eos_token, tokenizer.padding_side

('</s>', 'right')

In [16]:
tokenizer.padding_side = 'right'

In [17]:
model = AutoModelForCausalLM.from_pretrained(model_dir, torch_dtype=torch.float16, low_cpu_mem_usage=True) # , device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
model.to('cuda')

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [19]:
from transformers import set_seed

seed = 42 
set_seed(seed)

In [20]:
instruct = qa_df['input'][0]
instruct 

'당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.'

In [21]:
set(qa_df['input'])

{'당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.'}

In [22]:
def create_prompt_instruction(sample):
    return f"""### Instruction: 
    {sample['input']}

    ### 입력값 
    {sample['instruction']}

    ### 응답
    {sample['output']}
    """

In [23]:
create_prompt_instruction(qa_df.iloc[0])

'### Instruction: \n    당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.\n\n    ### 입력값 \n    다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞게 완성하세요. 문장은 이전 내용과 논리적으로 연결되어야 합니다.\n#텍스트: ‘고전문학교육’을 위해 ‘고전 시대의 문학 교육’의 양상을 살펴보는 일은 그 용어의 개념적인 면에서나 실제의 문화적 전통이라는 면에서나 소중한 일이다. 이에 대해 ‘과거 제도사’나 ‘학교 제도사’라는 측면에서 역사적 실상을 접근하려는 축과, 중세적 글쓰기의 양상을 간추려 그 문학 교육적 함의를 추론하려는 축으로 그 동안 적지 않은 연구 성과가 축적되었다. 전자가 교육제도의 변천에 중점을 두고 있다면, 후자는 중세 글쓰기의 현재적 가치를 추출해내는 데에 더 중점을 두고 있다. 그러나 모두 ‘고전 시대의 문학 교육’의 양상을 등한시 했다는 결함이 있다. 우리나라 중세시대에서 문학은 지금과는 다른 위치를 차지했다. 중세에는 문인이 사회적 헤게모니를 장악하고 있었고, 당연한 말이겠지만, 그들의 교양 중에서 문학은 다른 교양을 압도하는 주도적이고 본질적인 부분을 차지하고 있었다.\n\n    ### 응답\n    그래서 문학은 가옥의 배열이나 마을의 배치를 설명하거나, 귀신을 내쫓고 운명을 예견하는 일까지도 할 수 있는 것으로 여겨졌다.\n    '

In [24]:
def get_prompt(df):
    """
    """
    idx = randrange(len(df))
    sample = df.iloc[idx]
    return f"""### Instruction:
    {sample['input']}
    ### 입력값 
    {sample['instruction']}""", sample 

In [25]:
def get_response(prompt, sample):
    encoded_input = tokenizer(prompt, return_tensors='pt', add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')
    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
    decoded_output = tokenizer.batch_decode(generated_ids)
    return {
        'llm_response': decoded_output[0], 
        'reference': sample['output']
    }

In [26]:
qa_df.loc[29184]

input          당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변...
instruction    다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞...
output         이스라엘의 귀환권은 ‘유대인’이라는 민족적 정체성이 가장 중요한 기준으로 그 안에 ...
Name: 29184, dtype: object

In [27]:
test_prompt, sample = get_prompt(qa_df)
print(test_prompt)

### Instruction:
    당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.
    ### 입력값 
    다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞게 완성하세요. 문장은 이전 내용과 논리적으로 연결되어야 합니다.
#텍스트: 근대국가는 민족을 기반으로 구성된 개념이며, 근대국가로서 한국은 같은 문화와 역사, 언어를 공유하는 한민족의 개념에 기초해 한인들의 정체성을 유지하며 존속해 왔다. 세계화로 보편적 인류애가 강조되는 현대사회에서 배타적인 민족 관념과 자민족 우월주의는 경계되어야 하지만, 오랜 역사 속에서 형성해온 민족적 동질감은 한국의 소중한 자산 중의 하나이다. ‘민족’ 개념에서 유래한 ‘동포’ 개념역시 민족적 정체성을 대체로 공유하지만, 같은 공간에서 살지는 않고 국경 밖의 삶을 살고 있는 사람들을 뜻한다. 이 개념은 지금도 국가 발전을 위해서나 법적으로 소외받는 사람들을 위한 인류보편적인 차원에서도 의미있게 쓰이고 있다. 이스라엘은 1950년 국회에서 통과된 귀환법(Law of Return)에서 ‘유대인은 이민자의 신분으로 이스라엘에 정착할 권리가 있다’고 규정하고 있다. 그 후 개정되었던 귀환법은 타종교도와의 결혼으로 가족들이 이산되는 것을 방지하고, 선조의 신분 때문에 비유대인으로 분류되지 않도록 하기 위한 배려로, ‘유대인들, 그들의 비유대인 자녀들과 손자들, 배우자들과 자녀들, 손자들의 비유대인 배우자들까지 시민권을 당연한 권리’로 인정하였다. 이는 유대인이 오랜 기간 동안 전 세계에 흩어져 각 정주지역에 동화되어 왔기에 유대인 내부에서도 인종, 언어, 문화적 다양성으로 인하여 특정한 인종과 언어 문화적 정체성을 지정하지 않고, ‘유대인’이기만 하면 누구든지 귀환권을 인정받는 것이며, 유대인이 아니라면 귀환권을 부여하지 않다는 것을 의미한다. 이러한 법적 규정에 따라, 1948년 건국 이전부터 이스라엘 지역에 정주해 왔던 전체의 20%를 차지하고

In [28]:
response = get_response(test_prompt, sample)
print('LLM result: ')
print(response['llm_response'])
print('--' * 10)
print('Ground Truth:')
print(response['reference'])

Token indices sequence length is longer than the specified maximum sequence length for this model (1092 > 1024). Running this sequence through the model will result in indexing errors
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


LLM result: 
<s> ### Instruction:
    당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.
    ### 입력값 
    다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞게 완성하세요. 문장은 이전 내용과 논리적으로 연결되어야 합니다.
#텍스트: 근대국가는 민족을 기반으로 구성된 개념이며, 근대국가로서 한국은 같은 문화와 역사, 언어를 공유하는 한민족의 개념에 기초해 한인들의 정체성을 유지하며 존속해 왔다. 세계화로 보편적 인류애가 강조되는 현대사회에서 배타적인 민족 관념과 자민족 우월주의는 경계되어야 하지만, 오랜 역사 속에서 형성해온 민족적 동질감은 한국의 소중한 자산 중의 하나이다. ‘민족’ 개념에서 유래한 ‘동포’ 개념역시 민족적 정체성을 대체로 공유하지만, 같은 공간에서 살지는 않고 국경 밖의 삶을 살고 있는 사람들을 뜻한다. 이 개념은 지금도 국가 발전을 위해서나 법적으로 소외받는 사람들을 위한 인류보편적인 차원에서도 의미있게 쓰이고 있다. 이스라엘은 1950년 국회에서 통과된 귀환법(Law of Return)에서 ‘유대인은 이민자의 신분으로 이스라엘에 정착할 권리가 있다’고 규정하고 있다. 그 후 개정되었던 귀환법은 타종교도와의 결혼으로 가족들이 이산되는 것을 방지하고, 선조의 신분 때문에 비유대인으로 분류되지 않도록 하기 위한 배려로, ‘유대인들, 그들의 비유대인 자녀들과 손자들, 배우자들과 자녀들, 손자들의 비유대인 배우자들까지 시민권을 당연한 권리’로 인정하였다. 이는 유대인이 오랜 기간 동안 전 세계에 흩어져 각 정주지역에 동화되어 왔기에 유대인 내부에서도 인종, 언어, 문화적 다양성으로 인하여 특정한 인종과 언어 문화적 정체성을 지정하지 않고, ‘유대인’이기만 하면 누구든지 귀환권을 인정받는 것이며, 유대인이 아니라면 귀환권을 부여하지 않다는 것을 의미한다. 이러한 법적 규정에 따라, 1948년 건국 이전부터 이스라엘 지역에 정주해

In [38]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.3,
    target_modules = ['q_proj', 'v_proj', 'k_proj', 'o_proj'],
    r=64,
    bias='none', 
    task_type='CASUAL_LM'
)

In [39]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [40]:
qa_df.head(3)

,input,instruction,output
0,당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변...,"다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞...","그래서 문학은 가옥의 배열이나 마을의 배치를 설명하거나, 귀신을 내쫓고 운명을 예견..."
1,당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변...,"다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞...","그리고 기존의 연구들은 시장에서 관찰되는 변동성, 즉 관측변동성(observed v..."
2,당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변...,"다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞...","국내에서는 오상우가 번안한 검사가 이용된 바 있는데, 4가지의 소검사로 구성되어 있..."


In [41]:
dataset = Dataset.from_dict(qa_df).train_test_split(test_size=0.3)

In [42]:
train_dataset = dataset['train']
train_dataset

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 94846
})

In [43]:
args = TrainingArguments(
    output_dir='komistral_instruct_qa',
    num_train_epochs=100,
    per_device_train_batch_size=4,
    warmup_steps=0.03,
    logging_steps=10,
    save_strategy='epoch',
    learning_rate=2e-4,
    bf16=True,
    lr_scheduler_type='constant',
    disable_tqdm=True, 
    remove_unused_columns=False
)

In [44]:
max_seq_length = 1024 
trainer = SFTTrainer(
    model=model, 
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=create_prompt_instruction,
    args=args,
    train_dataset=train_dataset,
)

Generating train split: 0 examples [00:00, ? examples/s]

In [45]:
train_dataset['input'][:3]

['당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.',
 '당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.',
 '당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.']

In [46]:
trainer.train()
trainer.save_model('mistral_instruct_qa')

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacty of 23.99 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 53.03 GiB is allocated by PyTorch, and 916.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.save_model('mistral_instruct_qa')